# Imports

In [3]:
import os
import numpy as np
import tensorflow as tf

# Dataset Exploration

In [ ]:
# Directory that holds the data
DATA_DIR = '__________________'

# Subdirectories for each class
data_dir_dogs = os.path.join(DATA_DIR, '__________________')
data_dir_cats = os.path.join(DATA_DIR, '__________________')

# os.listdir returns a list containing all files under the given dir
print(f"There are {len(os.listdir(__________________))} .")
print(f"There are {len(os.listdir(__________________))} ")

# EarlyStoppingCallback

In [2]:
# Remember to inherit from the correct class
class EarlyStoppingCallback(tf.keras.callbacks.Callback):

    # Define the correct function signature for on_epoch_end method
    def on_epoch_end(self, epoch, logs=None):
        # Check if the accuracy is greater or equal to 0.95 and validation accuracy is greater or equal to 0.8
        if logs['accuracy'] >= 0.95 and logs['val_accuracy'] >= 0.80:
            print("\nReached 95% train accuracy and 80% validation accuracy, so cancelling training!")
            self.model.stop_training = True

# Create Model

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(150, 150, 3)),
        tf.keras.layers.Rescaling(1./255),
        # conv and pooling layers
        tf.keras.layers.Conv2D(32, (3,3)),
        tf.keras.layers.MaxPool2D((2,2)),
        #
        tf.keras.layers.Conv2D(64, (3,3)),
        tf.keras.layers.MaxPool2D((2,2)),
        #
        tf.keras.layers.Conv2D(128, (3,3)),
        tf.keras.layers.MaxPool2D((2,2)),
        # flatten
        tf.keras.layers.Flatten(),
        # hidden
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

model.compile( 
        optimizer=tf.keras.optimizers.RMSprop(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy']
)

# Train Model

In [ ]:
# Train the model and save the training history (this may take some time)
history = model.fit(
	training_dataset,
	epochs=15,
	validation_data=validation_dataset,
	callbacks = [EarlyStoppingCallback()]
)

# Plot Training and Validation Accuracy

# Get training and validation accuracies
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle('Training and validation accuracy')

for i, (data, label) in enumerate(zip([(acc, val_acc), (loss, val_loss)], ["Accuracy", "Loss"])):
    ax[i].plot(epochs, data[0], 'r', label="Training " + label)
    ax[i].plot(epochs, data[1], 'b', label="Validation " + label)
    ax[i].legend()
    ax[i].set_xlabel('epochs')

plt.show()